In [ ]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output, State, dash_table
import plotly.express as px
import dash_leaflet as dl
import dash_leaflet.express as dlx
import CRUDOperations
from jupyter_dash import JupyterDash
import os
from dotenv import load_dotenv
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()
MONGO_USERNAME = os.getenv('MONGO_USERNAME', 'aacuser')
MONGO_PASSWORD = os.getenv('MONGO_PASSWORD', 'aacpassword123')

# Grazioso Salvare Logo (Link or Path)
LOGO = "/home/jasonjourniga_snhu/Desktop/Grazioso.png"

# Initialize MongoDB connection
def initialize_db():
    """Initialize MongoDB connection with connection pooling."""
    try:
        shelter = CRUDOperations.CRUDOperations(
            MONGO_USERNAME, MONGO_PASSWORD, "nv-desktop-services.apporto.com", 32335, "AAC", "animals"
        )
        shelter.create_index([('animal_type', 1), ('breed', 1), ('outcome_type', 1)])
        return shelter
    except Exception as e:
        logger.error(f"Failed to initialize database: {e}")
        raise

shelter = initialize_db()

# Fetch and filter data
def fetch_data(query={}):
    """Fetch data from MongoDB and return filtered DataFrame."""
    try:
        data = pd.DataFrame.from_records(shelter.read(query))
        columns = ["animal_type", "breed", "color", "outcome_type", "outcome_subtype", "age_upon_outcome", "sex_upon_outcome", "location_lat", "location_long"]
        return data[columns] if not data.empty else pd.DataFrame(columns=columns)
    except Exception as e:
        logger.error(f"Error fetching data: {e}")
        return pd.DataFrame()

# Dash App Initialization
app = JupyterDash("Animal Center Dashboard", suppress_callback_exceptions=True)

# App Layout
app.layout = html.Div([
    # Logo and Title
    html.Div([
        html.Img(src=LOGO, style={"width": "200px"}),
        html.H1("Grazioso Salvare Animal Outcomes Dashboard", style={"textAlign": "center"}),
        html.P("Dashboard created by Jason Journigan", style={"textAlign": "center", "fontStyle": "italic"})
    ], style={"display": "flex", "flexDirection": "column", "alignItems": "center", "gap": "10px"}),

    # Filtering Options
    html.Div([
        html.Label("Filter by Animal Type:"),
        dcc.RadioItems(
            id="animal-type-filter",
            options=[
                {"label": "All", "value": "All"},
                {"label": "Dog", "value": "Dog"},
                {"label": "Cat", "value": "Cat"}
            ],
            value="All",
            inline=True
        ),
        html.Label("Filter by Rescue Type:"),
        dcc.Dropdown(
            id="rescue-type-filter",
            options=[
                {"label": "All", "value": "All"},
                {"label": "Water Rescue", "value": "Water Rescue"},
                {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"}
            ],
            value="All",
            placeholder="Select a Rescue Type",
            style={"marginBottom": "20px"}
        )
    ], style={"marginBottom": "20px"}),

    # Export Button
    html.Button("Export Filtered Data", id="export-button", n_clicks=0),
    dcc.Download(id="download-dataframe-csv"),

    # Data Table with Pagination
    dash_table.DataTable(
        id="animal-table",
        columns=[{"name": col, "id": col} for col in ["animal_type", "breed", "color", "outcome_type", "outcome_subtype", "age_upon_outcome", "sex_upon_outcome"]],
        page_size=10,
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "padding": "5px"}
    ),

    # Interactive Charts
    html.Div([
        dcc.Graph(id="outcome-chart"),
        dcc.Graph(id="age-chart")
    ], style={"display": "flex", "flexWrap": "wrap", "gap": "20px"}),

    # Map with Clustering
    html.Div([
        dl.Map(
            id="map",
            style={"width": "100%", "height": "500px"},
            center=[30.2672, -97.7431],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer"),
                dl.MarkerClusterGroup(id="marker-layer")
            ]
        )
    ])
])

# Callback for Updating Dashboard
@app.callback(
    [Output("animal-table", "data"),
     Output("outcome-chart", "figure"),
     Output("age-chart", "figure"),
     Output("marker-layer", "children")],
    [Input("animal-type-filter", "value"),
     Input("rescue-type-filter", "value")]
)
def update_dashboard(animal_type, rescue_type):
    """Update dashboard components based on filter inputs."""
    data_filtered = fetch_data()

    # Filter by Animal Type
    if animal_type != "All":
        data_filtered = data_filtered[data_filtered["animal_type"] == animal_type]

    # Advanced Filtering by Rescue Type
    if rescue_type == "Water Rescue":
        breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        data_filtered = data_filtered[
            (data_filtered["breed"].isin(breeds)) &
            (data_filtered["sex_upon_outcome"] == "Intact Female") &
            (data_filtered["age_upon_outcome"].str.contains("week", case=False, na=False))
        ]
    elif rescue_type == "Mountain or Wilderness Rescue":
        breeds = ["German Shepherd", "Alaskan Malamute", "Rottweiler"]
        data_filtered = data_filtered[
            (data_filtered["breed"].isin(breeds)) &
            (data_filtered["sex_upon_outcome"] == "Intact Male") &
            (data_filtered["age_upon_outcome"].str.contains("year", case=False, na=False))
        ]
    elif rescue_type == "Disaster or Individual Tracking":
        breeds = ["Bloodhound", "Coonhound", "Beagle"]
        data_filtered = data_filtered[
            (data_filtered["breed"].isin(breeds)) &
            (data_filtered["age_upon_outcome"].str.contains("month|year", case=False, na=False))
        ]

    # Outcome Chart (Pie Chart)
    outcome_fig = px.pie(
        data_filtered,
        names="outcome_type",
        title="Outcome Types",
        color_discrete_sequence=px.colors.sequential.RdBu
    )

    # Age Chart (Bar Chart)
    age_data = data_filtered["age_upon_outcome"].value_counts().reset_index()
    age_data.columns = ["Age", "Count"]
    age_fig = px.bar(
        age_data,
        x="Age",
        y="Count",
        title="Age Distribution",
        color="Count",
        color_continuous_scale=px.colors.sequential.Blues
    )

    # Map Markers with Clustering
    markers = []
    for _, row in data_filtered.iterrows():
        if pd.notnull(row["location_lat"]) and pd.notnull(row["location_long"]):
            markers.append(
                dl.Marker(
                    position=[row["location_lat"], row["location_long"]],
                    children=[
                        dl.Tooltip(f"{row['animal_type']} - {row['outcome_type']}"),
                        dl.Popup([html.P(f"Breed: {row['breed']}")])
                    ]
                )
            )

    table_data = data_filtered.drop(columns=["location_lat", "location_long"]).to_dict('records')
    return table_data, outcome_fig, age_fig, markers

# Callback for Exporting Data
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    State("animal-type-filter", "value"),
    State("rescue-type-filter", "value"),
    prevent_initial_call=True
)
def export_data(n_clicks, animal_type, rescue_type):
    """Export filtered data as CSV."""
    data_filtered = fetch_data()

    if animal_type != "All":
        data_filtered = data_filtered[data_filtered["animal_type"] == animal_type]

    if rescue_type == "Water Rescue":
        breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        data_filtered = data_filtered[
            (data_filtered["breed"].isin(breeds)) &
            (data_filtered["sex_upon_outcome"] == "Intact Female") &
            (data_filtered["age_upon_outcome"].str.contains("week", case=False, na=False))
        ]
    elif rescue_type == "Mountain or Wilderness Rescue":
        breeds = ["German Shepherd", "Alaskan Malamute", "Rottweiler"]
        data_filtered = data_filtered[
            (data_filtered["breed"].isin(breeds)) &
            (data_filtered["sex_upon_outcome"] == "Intact Male") &
            (data_filtered["age_upon_outcome"].str.contains("year", case=False, na=False))
        ]
    elif rescue_type == "Disaster or Individual Tracking":
        breeds = ["Bloodhound", "Coonhound", "Beagle"]
        data_filtered = data_filtered[
            (data_filtered["breed"].isin(breeds)) &
            (data_filtered["age_upon_outcome"].str.contains("month|year", case=False, na=False))
        ]

    return dcc.send_data_frame(data_filtered.to_csv, "filtered_animals.csv")

# Run the App
if __name__ == "__main__":
    app.run_server(debug=True, port=8053)